In [14]:
import pandas as pd
import os
from datetime import datetime

In [15]:
def carregar_dados(arquivo):
    if not os.path.exists(arquivo):
        raise FileNotFoundError(f"O arquivo {arquivo} não foi encontrado.")
    df = pd.read_csv(arquivo)
    print(f"Carregado {arquivo}: {len(df)} linhas")
    return df

# Função para verificar se um DataFrame está vazio e mostrar informações sobre as colunas
def verificar_df(df, nome):
    if df.empty:
        print(f"AVISO: O DataFrame {nome} está vazio!")
    else:
        print(f"{nome} contém {len(df)} linhas")
        print(f"Colunas: {df.columns.tolist()}")
        print(f"Tipos de dados:\n{df.dtypes}")
        print(f"Valores nulos:\n{df.isnull().sum()}")
        if 'data' in df.columns:
            print(f"Amostra de datas:\n{df['data'].head()}")
        elif 'dt_diagnostico_sintoma' in df.columns:
            print(f"Amostra de datas de diagnóstico:\n{df['dt_diagnostico_sintoma'].head()}")
    print("--------------------")

In [16]:
# Carregando os dados
try:
    df_origem = carregar_dados('origem.csv')
    df_municipio = carregar_dados('d_municipio.csv')
    df_tempo = carregar_dados('d_tempo.csv')
except FileNotFoundError as e:
    print(f"Erro ao carregar arquivos: {e}")
    exit(1)



Carregado origem.csv: 27820 linhas
Carregado d_municipio.csv: 5598 linhas
Carregado d_tempo.csv: 14975 linhas


In [17]:
# Converte 'dt_diagnostico_sintoma' para datetime
df_origem['dt_diagnostico_sintoma'] = pd.to_datetime(df_origem['dt_diagnostico_sintoma'], errors='coerce')

# Filtra os dados conforme os critérios especificados
df_filtrado = df_origem[
    ((df_origem['tp_entrada'].isin([1, 6])) |
    (df_origem['tp_entrada'].isna())) &
    (df_origem['tp_forma'].isin([1, 3])) &
    (df_origem['tp_situacao_encerramento'] != 6) &
    (df_origem['dt_diagnostico_sintoma'].dt.year >= 2021) &  # Agora podemos acessar o ano corretamente
    (df_origem['co_uf_residencia_atual'] == 52)  # Código UF de Goiás
]

# Após o filtro, formata a coluna 'dt_diagnostico_sintoma' como string
df_filtrado['data'] = df_filtrado['dt_diagnostico_sintoma'].dt.strftime('%d/%m/%Y')

verificar_df(df_filtrado, "df_filtrado após correção de data")


df_filtrado após correção de data contém 3320 linhas
Colunas: ['nu_notificacao', 'tp_entrada', 'tp_pop_liberdade', 'tp_pop_rua', 'tp_forma', 'dt_nascimento', 'dt_diagnostico_sintoma', 'tp_situacao_encerramento', 'co_uf_residencia_atual', 'co_municipio_residencia_atual', 'data']
Tipos de dados:
nu_notificacao                            int64
tp_entrada                              float64
tp_pop_liberdade                        float64
tp_pop_rua                              float64
tp_forma                                float64
dt_nascimento                            object
dt_diagnostico_sintoma           datetime64[ns]
tp_situacao_encerramento                float64
co_uf_residencia_atual                  float64
co_municipio_residencia_atual           float64
data                                     object
dtype: object
Valores nulos:
nu_notificacao                     0
tp_entrada                         0
tp_pop_liberdade                  46
tp_pop_rua                        51


C:\Users\Filipe_Cortes\AppData\Local\Temp\ipykernel_26568\692620896.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['data'] = df_filtrado['dt_diagnostico_sintoma'].dt.strftime('%d/%m/%Y')


In [18]:
# Agrega os dados (agora utilizando a coluna 'data')
df_agregado = df_filtrado.groupby(['co_municipio_residencia_atual', 'data']).size().reset_index(name='quantidade')

verificar_df(df_agregado, "df_agregado")

# Adiciona as informações dos municípios
df_final = df_agregado.merge(df_municipio[['dmun_codibge', 'dmun_municipio']], 
                             left_on='co_municipio_residencia_atual', 
                             right_on='dmun_codibge', 
                             how='left')

verificar_df(df_final, "df_final após merge com municípios")

# Converte a coluna 'date_medium' para datetime, assumindo que o dia vem antes do mês
df_tempo['data'] = pd.to_datetime(df_tempo['date_medium'], dayfirst=True, errors='coerce').dt.strftime('%d/%m/%Y')

verificar_df(df_tempo, "df_tempo após formatação de data")

df_agregado contém 2842 linhas
Colunas: ['co_municipio_residencia_atual', 'data', 'quantidade']
Tipos de dados:
co_municipio_residencia_atual    float64
data                              object
quantidade                         int64
dtype: object
Valores nulos:
co_municipio_residencia_atual    0
data                             0
quantidade                       0
dtype: int64
Amostra de datas:
0    04/09/2023
1    06/08/2022
2    11/09/2023
3    12/05/2023
4    13/09/2023
Name: data, dtype: object
--------------------
df_final após merge com municípios contém 2842 linhas
Colunas: ['co_municipio_residencia_atual', 'data', 'quantidade', 'dmun_codibge', 'dmun_municipio']
Tipos de dados:
co_municipio_residencia_atual    float64
data                              object
quantidade                         int64
dmun_codibge                       int64
dmun_municipio                    object
dtype: object
Valores nulos:
co_municipio_residencia_atual    0
data                             0


In [19]:
# Adiciona as informações temporais
df_final = df_final.merge(df_tempo[['data', 'month_name', 'year4']], 
                          on='data', 
                          how='left')

# Verifique o DataFrame após o merge
verificar_df(df_final, "df_final após merge com tempo")

# Formata a coluna de mês
df_final['mes'] = df_final['month_name'].fillna('Mês desconhecido') + ' de ' + df_final['year4'].fillna('Ano desconhecido').astype(str)

# Seleciona e renomeia as colunas finais
df_saida = df_final[['dmun_municipio', 'mes', 'quantidade']]
df_saida.columns = ['municipio', 'mes', 'quantidade']

# Ordena o DataFrame
df_saida = df_saida.sort_values(['municipio', 'mes'])

verificar_df(df_saida, "df_saida final")

# Salva o resultado
df_saida.to_csv('saida.csv', index=False)

# Verifica se o arquivo foi criado e não está vazio
if os.path.exists('saida.csv'):
    tamanho = os.path.getsize('saida.csv')
    print(f"Arquivo saida.csv criado com sucesso. Tamanho: {tamanho} bytes")
    if tamanho == 0:
        print("AVISO: O arquivo saida.csv está vazio!")
else:
    print("ERRO: O arquivo saida.csv não foi criado!")

print("Processo ETL concluído.")

df_final após merge com tempo contém 2842 linhas
Colunas: ['co_municipio_residencia_atual', 'data', 'quantidade', 'dmun_codibge', 'dmun_municipio', 'month_name', 'year4']
Tipos de dados:
co_municipio_residencia_atual    float64
data                              object
quantidade                         int64
dmun_codibge                       int64
dmun_municipio                    object
month_name                        object
year4                              int64
dtype: object
Valores nulos:
co_municipio_residencia_atual    0
data                             0
quantidade                       0
dmun_codibge                     0
dmun_municipio                   0
month_name                       0
year4                            0
dtype: int64
Amostra de datas:
0    04/09/2023
1    06/08/2022
2    11/09/2023
3    12/05/2023
4    13/09/2023
Name: data, dtype: object
--------------------
df_saida final contém 2842 linhas
Colunas: ['municipio', 'mes', 'quantidade']
Tipos de dados:
